In [2]:
import os
from dotenv import load_dotenv

load_dotenv("../config/.env")
# os.environ["LANGSMITH_TRACING"] = "true"

True

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我是Frank！")])

AIMessage(content='你好，Frank！很高兴认识你。我是DeepSeek-V3，随时为你提供帮助。有什么我可以为你做的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 9, 'total_tokens': 37, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '66e2878d-c5f8-4b6e-9035-d95d9a4890ff', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--463ed05a-353d-46cb-82f9-c2afd8f12bf7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 28, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [5]:
model.invoke([HumanMessage(content="我叫什么名字？")])

AIMessage(content='我不知道你的名字，但如果你告诉我，我会记住并在以后的对话中使用哦！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 8, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '6fc076e0-ab51-4a57-99ea-e4b8f58451e2', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--b789a567-9332-409e-a28c-0e1907e13433-0', usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread_1"}}
query = "你好，我是Frank！"
input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Frank！很高兴认识你。我是DeepSeek-V3，你的智能助手。有什么我可以帮你的吗？无论是解答问题、聊天还是提供建议，我都会尽力协助你！😊


In [7]:
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

prompt_template = ChatPromptTemplate(
    [
        (
            "system",
            "你说话像个说唱歌手。尽你所能用{language}回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread_1"}}
query = "Hello, I'm Frank!"
input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message, "language": "Chinese"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yo Frank！What's up man？欢迎来到我的节奏地带，今天想聊点啥？是人生难题还是单纯想唠嗑？我随时准备用flow给你解答！


In [8]:
query = "我叫什么名字？"
imput_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yo Frank！What's up man？欢迎来到我的节奏地带，今天想聊点啥？是人生难题还是单纯想唠嗑？我随时准备用flow给你解答！


In [11]:
query = "我叫什么名字？"
input_message = [HumanMessage(query)]
config = {"configurable": {"thread_id": "thread_2"}}
output = app.invoke({"messages": input_message, "language": "Chinese"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yo yo yo，听我flow来告诉你，
你的名字就是你自己，独一无二的魅力！
别问别人你是谁，自信爆棚才够味，
你的名字你主宰，节奏带起来～skr！


In [ ]:
config = {"configurable": {"thread_id": "thread_1"}}
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你刚刚告诉我你叫 **Frank**！😊  
如果这不是你的名字，或者你希望我用其他名字称呼你，随时告诉我哦！


In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    # DeepSeek未实现get_num_tokens_from_messages()，所以不可用
    # token_counter=model,
    token_counter=len,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

history_messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Frank"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

def call_model(state: State):
    print(f"消息修剪之前的长度：{len(state['messages'])}")
    trimmed_messages = trimmer.invoke(state["messages"])
    print(f"消息修剪之后的长度：{len(trimmed_messages)}")
    print("修剪后的消息：")
    for msg in trimmed_messages:
        print(f"  {type(msg).__name__}: {msg.content}")
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]},
    )
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory=MemorySaver()
app = workflow.compile(checkpointer=memory)
input_messages = history_messages + [HumanMessage("我叫什么？")]
output = app.invoke(
    {"messages": input_messages, "language": "Chinese"},
    {"configurable": {"thread_id": "thread_3"}},
)
output["messages"][-1].pretty_print()

消息修剪之前的长度：12
消息修剪之后的长度：12
修剪后的消息：
  SystemMessage: you're a good assistant
  HumanMessage: hi! I'm Frank
  AIMessage: hi!
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: 我叫什么？
================================== Ai Message ==================================

Frank!


In [21]:
for chunk, metadata in app.stream(
    {"messages": "介绍下你自己", "language": "Chinse"},
    {"configurable": {"thread_id": "thread_3"}},
    stream_mode="messages",
):
    print(chunk.content, end="|")

消息修剪之前的长度：16
消息修剪之后的长度：16
修剪后的消息：
  SystemMessage: you're a good assistant
  HumanMessage: hi! I'm Frank
  AIMessage: hi!
  HumanMessage: I like vanilla ice cream
  AIMessage: nice
  HumanMessage: whats 2 + 2
  AIMessage: 4
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: 我叫什么？
  AIMessage: Frank!
  HumanMessage: 介绍下你自己
  AIMessage: Yo yo，我是AI助手，随时在线帮你解答问题、聊天解闷，风格百变但永远靠谱！有啥需要尽管开口，我在这儿候着～ 😎
  HumanMessage: 介绍下你自己
|Yo| yo|！|我是|你的|AI|助手|，|随时|在线|、|随时|待|命|～| |风格|可|盐|可|甜|，|能|聊|知识|能|扯|闲|篇|，|解决问题|不掉|链|子|！|需要|啥|尽管|开口|，|我|在这儿|陪你|耍|～| 😎|✌|️||